In [2]:
import yaml

In [41]:
test_yaml ="""
code: fipi
locales: ["en", "ru"]
en:  # title, decsriptions, instruction
  - "FIPI"
  - "Five-Item Personality Inventory (for testing only)" 
  - "Answer all the questions (there's only 5 of them!)"
ru: # title, decsriptions, instruction
  - "ПИПИ"
  - "Краткий тест личности (для тестов)"
  - "Ответьте на все вопросы (их всего 5!)"
type: sten
steps: 9
randomize: false
question:
  en: "How this is about you?" # content
  ru: "Оцените, насколько вам это подходит?" #content
scales: 
  extraversion: 
    en: ["Extraversion", "Extraversion scale"] # title, description
    ru: ["Экстраверсия", "Шкала экстраверсии"] # title, description
  agreeableness:
    en: ["Agreeableness", "Agreeableness scale"] # title, description
    ru: ["Доброжелательность", "Шкала доброжелательности"] # title, description
  conscientiousness:
    en: ["Conscientiousness", "Conscientiousness scale"] # title, description
    ru: ["Сознательность", "Шкала сознательнольсти"] # title, description
  stability:
    en: ["Emotional Stability", "Emotional stability scale"] # title, description
    ru: ["Эмоциональная стабильность", "Шкала эмоциональной стабильности"] # title, description
  openness:
    en: ["Openness to experience", "Openness to experience scale"] # title, description
    ru: ["Открытость новому опыту", "Шкала открытости новому опыту"] # title, description
items:
  - en: "Extraverted, enthusiastic." # content
    ru: "Экстраверт, полный энтузиазма." # content
    scale": "extraversion"
  - en: "Critical, quarrelsome." # content
    ru: "Критичный, придирчивый." # content
    scale: "-agreeableness" # reverse
  - en: "Dependable, self-disciplined." # content
    ru: "Надежный, дисциплинированный." # content
    scale: "conscientiousness"
  - en: "Anxious, easily upset." # content
    ru: "Тревожный, легко расстраиваемый." # content
    scale: "-stability"
  - en: "Open to new experiences, complex." # content
    ru: "Открытый опыту, многоуровневый." # content
    scale: "openness"
"""

In [42]:
t = yaml.safe_load(test_yaml)
t

{'code': 'fipi',
 'locales': ['en', 'ru'],
 'en': ['FIPI',
  'Five-Item Personality Inventory (for testing only)',
  "Answer all the questions (there's only 5 of them!)"],
 'ru': ['ПИПИ',
  'Краткий тест личности (для тестов)',
  'Ответьте на все вопросы (их всего 5!)'],
 'type': 'sten',
 'steps': 9,
 'randomize': False,
 'question': {'en': 'How this is about you?',
  'ru': 'Оцените, насколько вам это подходит?'},
 'scales': {'extraversion': {'en': ['Extraversion', 'Extraversion scale'],
   'ru': ['Экстраверсия', 'Шкала экстраверсии']},
  'agreeableness': {'en': ['Agreeableness', 'Agreeableness scale'],
   'ru': ['Доброжелательность', 'Шкала доброжелательности']},
  'conscientiousness': {'en': ['Conscientiousness', 'Conscientiousness scale'],
   'ru': ['Сознательность', 'Шкала сознательнольсти']},
  'stability': {'en': ['Emotional Stability', 'Emotional stability scale'],
   'ru': ['Эмоциональная стабильность', 'Шкала эмоциональной стабильности']},
  'openness': {'en': ['Openness to ex

In [49]:
translations = [
    {
        "locale": l,
        "title": t[l][0],
        "description": t[l][1],
        "instruction": t[l][2],
    } for l in t["locales"]
]

display = {
    "questionsPerPage": 1,
    "randomizeOrder": t["randomize"],
    
}

scales = [
    {
        "code": f'{t["code"]}-{code}',
        "type": t["type"],
        "translations": [
            {
                "locale": loc,
                "title": trans,
                "description": trans,
            } for loc, trans in s.items()
        ],
    } for code, s in t["scales"].items()
]

result = {
    "code": t["code"],
    "published": False,
    "availableLocales": t["locales"],
    "forceUpdate": False,
    "translations": translations,
    "scales": scales,
    "display": display,
}

print(yaml.dump(result, sort_keys=False, allow_unicode=True))
result

code: fipi
published: false
availableLocales:
- en
- ru
forceUpdate: false
translations:
- locale: en
  title: FIPI
  description: Five-Item Personality Inventory (for testing only)
  instruction: Answer all the questions (there's only 5 of them!)
- locale: ru
  title: ПИПИ
  description: Краткий тест личности (для тестов)
  instruction: Ответьте на все вопросы (их всего 5!)
scales:
- code: fipi-extraversion
  type: sten
  translations:
  - locale: en
    title: &id001
    - Extraversion
    - Extraversion scale
    description: *id001
  - locale: ru
    title: &id002
    - Экстраверсия
    - Шкала экстраверсии
    description: *id002
- code: fipi-agreeableness
  type: sten
  translations:
  - locale: en
    title: &id003
    - Agreeableness
    - Agreeableness scale
    description: *id003
  - locale: ru
    title: &id004
    - Доброжелательность
    - Шкала доброжелательности
    description: *id004
- code: fipi-conscientiousness
  type: sten
  translations:
  - locale: en
    title:

{'code': 'fipi',
 'published': False,
 'availableLocales': ['en', 'ru'],
 'forceUpdate': False,
 'translations': [{'locale': 'en',
   'title': 'FIPI',
   'description': 'Five-Item Personality Inventory (for testing only)',
   'instruction': "Answer all the questions (there's only 5 of them!)"},
  {'locale': 'ru',
   'title': 'ПИПИ',
   'description': 'Краткий тест личности (для тестов)',
   'instruction': 'Ответьте на все вопросы (их всего 5!)'}],
 'scales': [{'code': 'fipi-extraversion',
   'type': 'sten',
   'translations': [{'locale': 'en',
     'title': ['Extraversion', 'Extraversion scale'],
     'description': ['Extraversion', 'Extraversion scale']},
    {'locale': 'ru',
     'title': ['Экстраверсия', 'Шкала экстраверсии'],
     'description': ['Экстраверсия', 'Шкала экстраверсии']}]},
  {'code': 'fipi-agreeableness',
   'type': 'sten',
   'translations': [{'locale': 'en',
     'title': ['Agreeableness', 'Agreeableness scale'],
     'description': ['Agreeableness', 'Agreeableness

In [ ]:
code: fipi
published: true
availableLocales: ["en", "ru"]
forceUpdate: true
translations:
  - locale: en
    title: FIPI
    description: Five-Item Personality Inventory (for testing only)
    instruction: Answer all the questions (there's only 5 of them!)
  - locale: ru
    title: ТИПИ
    description: Краткий тест личности (для тестов)
    instruction: Ответьте на все вопросы (их всего 5!)
scales: 
  - code: fipi-scale-1-extraversion
    type: mean
    translations:
      - locale: en
        title: Extraversion
        description: Extraversion scale
      - locale: ru
        title: ТИПИ-Экстраверсия
        description: Шкала экстраверсии
    items:
      - code: fipi-item-ex-1
        steps: 9
        reverse: false
        translations: 
          - locale: en
            content: Extraverted, enthusiastic.
          - locale: ru
            content: Экстраверт, полный энтузиазма.
  - code: fipi-scale-2-agreeableness
    type: mean
    translations:
      - locale: en
        title: FIPI-Agreeableness
        description: Agreeableness scale
      - locale: ru
        title: ТИПИ-Доброжелательность
        description: Шкала доброжелательности
    items:
      - code: fipi-item-agr-1
        steps: 9
        reverse: true
        translations: 
          - locale: en
            content: Critical, quarrelsome.
          - locale: ru
            content: Критичный, придирчивый.
  - code: fipi-scale-3-conscientiousness
    type: mean
    translations:
      - locale: en
        title: FIPI-Conscientiousness
        description: Conscientiousness scale
      - locale: ru
        title: ТИПИ-Сознательность
        description: Шкала сознательнольсти
    items:
      - code: fipi-item-cons-1
        steps: 9
        reverse: false
        translations: 
          - locale: en
            content: Dependable, self-disciplined.
          - locale: ru
            content: Надежный, дисциплинированный.
  - code: fipi-scale-4-stability
    type: mean
    translations:
      - locale: en
        title: FIPI-Emotional Stability
        description: Emotional Stability scale
      - locale: ru
        title: ТИПИ-Эмоциональная стабильность
        description: Шкала эмоциональной стабильности
    items:
      - code: fipi-item-stab-1
        steps: 9
        reverse: true
        translations: 
          - locale: en
            content: Anxious, easily upset.
          - locale: ru
            content: Тревожный, легко расстраиваемый.
  - code: fipi-scale-5-openness
    type: mean
    translations:
      - locale: en
        title: FIPI-Openness to Experience
        description: Openness to Experience scale
      - locale: ru
        title: ТИПИ-Открытость новому опыту
        description: Шкала открытости новому опыту
    items:
      - code: fipi-item-op-1
        steps: 9
        reverse: false
        translations: 
          - locale: en
            content: Open to new experiences, complex.
          - locale: ru
            content: Открытый опыту, многоуровневый.
questions: 
  - code: fipi-question-1
    order: 10
    type: simple
    items: 
      - code: fipi-item-op-1
    translations: 
      - locale: en
        content: "How this is about`l you?"
      - locale: ru
        content: "Оцените, насколько вам это подходит?"
  - code: fipi-question-2
    order: 20
    type: simple
    items: 
      - code: fipi-item-stab-1
    translations: 
      - locale: en
        content: "How this is about you?"
      - locale: ru
        content: "Оцените, насколько вам это подходит?"
  - code: fipi-question-3
    order: 30
    type: simple
    items: 
      - code: fipi-item-cons-1
    translations: 
      - locale: en
        content: "How this is about you?"
      - locale: ru
        content: "Оцените, насколько вам это подходит?"
  - code: fipi-question-4
    order: 40
    type: simple
    items: 
      - code: fipi-item-agr-1
    translations: 
      - locale: en
        content: "How this is about you?"
      - locale: ru
        content: "Оцените, насколько вам это подходит?"
  - code: fipi-question-5
    order: 50
    type: simple
    items: 
      - code: fipi-item-ex-1
    translations: 
      - locale: en
        content: "How this is about you?"
      - locale: ru
        content: "Оцените, насколько вам это подходит?"
generate: 
  method: none
display:
  questionsPerPage: 2
  randomizeOrder: false